In [24]:
#%pip install googlemaps geopy networkx matplotlib

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
import pandas as pd
import googlemaps
import networkx as nx
from geopy.distance import geodesic
import time
import requests

In [26]:
cities = pd.read_csv('mk.csv')

In [27]:
cities.head()

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Skopje,41.9961,21.4317,North Macedonia,MK,Skopje,primary,526502.0,526502.0
1,Kumanovo,42.1322,21.7144,North Macedonia,MK,Kumanovo,admin,70842.0,70842.0
2,Bitola,41.0319,21.3347,North Macedonia,MK,Bitola,admin,69287.0,69287.0
3,Prilep,41.3464,21.5542,North Macedonia,MK,Prilep,admin,63308.0,63308.0
4,Tetovo,42.0103,20.9714,North Macedonia,MK,Tetovo,admin,63176.0,63176.0


In [ ]:

cities_dict = {row.city: (row.lat, row.lng) for row in cities.itertuples(index=False)}
coord_str = ";".join(f"{lng},{lat}" for lat, lng in cities_dict.values())

url = f"http://router.project-osrm.org/table/v1/driving/{coord_str}?annotations=duration"

print("Requesting travel times from OSRM...")

response = requests.get(url)
response.raise_for_status()
data = response.json()

durations = data["durations"]

city_names = list(cities_dict.keys())
matrix = pd.DataFrame(durations, index=city_names, columns=city_names)

matrix = matrix / 60

matrix.to_csv("macedonia_travel_time_matrix.csv", float_format="%.1f")

print("\n✅ Travel time matrix (in minutes):")
print(matrix.round(1))
print("\nSaved as 'macedonia_travel_time_matrix.csv'")


Requesting travel times from OSRM...

✅ Travel time matrix (in minutes):
                     Skopje  Kumanovo  Bitola  Prilep  Tetovo   Štip  Veles  \
Skopje                  0.0      37.7   167.5   120.0    41.3   61.4   58.8   
Kumanovo               36.7       0.0   164.0   116.5    64.4   54.9   55.3   
Bitola                157.2     152.9     0.0    52.9   175.9  152.4  122.5   
Prilep                108.9     104.6    53.0     0.0   135.4  104.1   74.2   
Tetovo                 43.2      66.5   176.2   148.4     0.0   90.2   87.2   
...                     ...       ...     ...     ...     ...    ...    ...   
Bogovinje              53.9      77.3   167.7   150.5    13.7  100.9   97.9   
Ilinden                21.3      25.1   158.8   111.2    48.9   48.8   50.0   
Lipkovo                41.2      16.1   176.5   129.0    65.8   67.4   67.8   
Makedonska Kamenica   102.0      94.7   191.5   144.0   129.6   57.3   84.1   
Resen                 187.7     183.4    37.7    83.4   18